In [650]:
# Fixed dependencies - do not remove or change.
import pytest
import pandas as pd
import numpy as np
import tensorflow as tf
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [651]:
#tidy up the file and replace the ranged data with averages! makes it all tidier.
#issues with the CSV file this makes it all work but isnt overly efficient sorry
text = open('/content/gdrive/MyDrive/breast-cancer.csv', 'r')
text = ''.join([i for i in text]) \
    .replace("Oct-14", "10-14")
text = ''.join([i for i in text]) \
    .replace("3-May", "3-5")
text = ''.join([i for i in text]) \
    .replace("6-Aug", "6-8")
text = ''.join([i for i in text]) \
    .replace("9-Nov", "9-11")
text = ''.join([i for i in text]) \
    .replace("40-49", "45")
text = ''.join([i for i in text]) \
    .replace("50-59", "55")
text = ''.join([i for i in text]) \
    .replace("60-69", "65")
text = ''.join([i for i in text]) \
    .replace("30-39", "35")
text = ''.join([i for i in text]) \
    .replace("70-79", "75")
text = ''.join([i for i in text]) \
    .replace("0-2", "1")
text = ''.join([i for i in text]) \
    .replace("3-5", "4")
text = ''.join([i for i in text]) \
    .replace("15-17", "16")
text = ''.join([i for i in text]) \
    .replace("6-8", "7")
text = ''.join([i for i in text]) \
    .replace("9-11", "10")
text = ''.join([i for i in text]) \
    .replace("15-19", "17")
text = ''.join([i for i in text]) \
    .replace("35-39", "37")
text = ''.join([i for i in text]) \
    .replace("30-34", "32")
text = ''.join([i for i in text]) \
    .replace("25-29", "27")
text = ''.join([i for i in text]) \
    .replace("40-44", "42")
text = ''.join([i for i in text]) \
    .replace("10-14", "12")
text = ''.join([i for i in text]) \
    .replace("0-4", "2")
text = ''.join([i for i in text]) \
    .replace("20-24", "22")
text = ''.join([i for i in text]) \
    .replace("15-19", "17")
text = ''.join([i for i in text]) \
    .replace("45-49", "47")
text = ''.join([i for i in text]) \
    .replace("50-54", "52")
text = ''.join([i for i in text]) \
    .replace("214", "22")
text = ''.join([i for i in text]) \
    .replace("?", "no")
x = open('/content/gdrive/MyDrive/breast-cancer.csv', 'w')
x.writelines(text)
print(x)
x.close()

<_io.TextIOWrapper name='/content/gdrive/MyDrive/breast-cancer.csv' mode='w' encoding='UTF-8'>


In [652]:
# Import data
def import_local_data(file_path):
    raw_df = pd.read_csv(file_path)
    return raw_df

In [653]:
local_file_path = '/content/gdrive/MyDrive/breast-cancer.csv'

In [654]:
raw_data = import_local_data(local_file_path)

In [655]:
raw_data.shape

(286, 10)

In [656]:
raw_data.head() #display the data, see comments below.

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat,Class
0,45,premeno,17,1,yes,3,right,left_up,no,recurrence-events
1,55,ge40,17,1,no,1,right,central,no,no-recurrence-events
2,55,ge40,37,1,no,2,left,left_low,no,recurrence-events
3,45,premeno,37,1,yes,3,right,left_low,yes,no-recurrence-events
4,45,premeno,32,4,yes,2,left,right_up,no,recurrence-events


In [657]:
raw_data.dtypes
#The data has 10 columns and 286 rows
# It is almost all object data with some int64
#The classifier column is whether the breast cancer is recurrent or not.
#Age: age of the patient at the time of diagnosis;
#Menopause: whether the patient is pre- or postmenopausal at time of diagnosis;
#Tumor size: the greatest diameter (in mm) of the excised tumor;
#Inv-nodes: the number (range 0 - 39) of axillary lymph nodes that contain metastatic breast cancer visible on histological examination;
#Node caps: if it is in the lymph nodes
#Degree of malignancy: the histological grade (range 1-3) of the tumor
#Breast: breast cancer may obviously occur in either breast;
#Breast quadrant: the breast may be divided into four quadrants
#Irradiation: have they had radiation therapy? Y or N

#The data was tidied up as with the text replace file to make it more ML friendly
#The age, tumor size and inv-nodes were averaged in their range
#the incorrect values were changed e.g. Oct-16 to 10-16

# SOME THINGS TO NOTE ABOUT THE DATA 
# 1. it appears there is maybe some correlation between inv-nodes and recurrence
# 2. One hot encoding was used on the rest of the data to make it from object format to ML friend
# 3. The breast and breast quadrant values dont agree! Therefore I will ignore the breast column.
# 4. There are significantly more non-recurrence than recurrence events 
# which could affect the quality of the model and predictions.

age             int64
menopause      object
tumor-size     object
inv-nodes      object
node-caps      object
deg-malig       int64
breast         object
breast-quad    object
irradiat       object
Class          object
dtype: object

In [658]:
del raw_data['breast']
raw_data.head()

,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast-quad,irradiat,Class
0,45,premeno,17,1,yes,3,left_up,no,recurrence-events
1,55,ge40,17,1,no,1,central,no,no-recurrence-events
2,55,ge40,37,1,no,2,left_low,no,recurrence-events
3,45,premeno,37,1,yes,3,left_low,yes,no-recurrence-events
4,45,premeno,32,4,yes,2,right_up,no,recurrence-events


In [659]:
#Dealing with any missing data! Only applies to the numeric data and will replace null values with the mean of column.
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer.fit(X[:, 0:1])
X[:, 0:1] = imputer.transform(X[:, 0:1])
imputer.fit(X[:, 2:4])
X[:, 2:4] = imputer.transform(X[:, 2:4])
imputer.fit(X[:, 5:6])
X[:, 5:6] = imputer.transform(X[:, 5:6])

In [660]:
# Split your data so that you can test the effectiveness of your model
X = raw_data.iloc[:, :-1].values #get all except last column
y = raw_data.iloc[:, -1].values #get last column / Classifier

#My processing HAS to be done like this.
#Using OHE, if you split the data, it DOES NOT generate the same number of columns because it sees a different number of variables
#in each test set so the columns do not equate if split the data and then process!
#Spent so many hours on this bit and why it wouldnt work so please understand! thanks.
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
X = np.array(ct.fit_transform(X)) 
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [6])], remainder='passthrough')
X = np.array(ct.fit_transform(X)) 
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [9])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [20])], remainder='passthrough')
X = np.array(ct.fit_transform(X)) 
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [27])], remainder='passthrough')
X = np.array(ct.fit_transform(X)) 
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [29])], remainder='passthrough')
X = np.array(ct.fit_transform(X)) 
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [32])], remainder='passthrough')
X = np.array(ct.fit_transform(X)) 
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [38])], remainder='passthrough')
X = np.array(ct.fit_transform(X)) 

#Now encode the y values.
le = LabelEncoder()
y = le.fit_transform(y)

In [661]:
from sklearn.model_selection import train_test_split #create training set and test set
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)
#it is randomly split here so the processing before has no impact.

In [662]:
# Populate preprocess_training_data and preprocess_test_data to preprocess data.
# You must process test and train separately so your model does not accidently gain information that a model wouldnt have in reality and therefore get better predictions
class Module4_Model:
    def __init__(self):
        self.model = None
        
    def preprocess_training_data(self, training_df):
      from sklearn.preprocessing import StandardScaler
      sc = StandardScaler()
      x_train = training_df
      x_train = sc.fit_transform(x_train)
#please see above comments for explanation of why it is not done here.
      processed_df = x_train
      return processed_df

    def preprocess_test_data(self, test_df):
 #please see above comments for explanation of why it is not done here.
      x_test = test_df
      x_test = sc.transform(x_test)
      processed_df = x_test
      return processed_df

In [663]:
# Dont change
my_model = Module4_Model()

In [664]:
# Dont change
x_train_processed = my_model.preprocess_training_data(x_train)

In [665]:
# Dont change
x_test_processed = my_model.preprocess_test_data(x_test)

In [666]:
ann = tf.keras.models.Sequential() #create an ANN with sequential layers!

In [667]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))  #Add the first hidden layer with 6 neurons

In [668]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu')) #Add the 2nd hidden layer and give it 6 hidden nuerons in this layer!

In [669]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid')) #Add the output layer

In [670]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy']) #Compile the ann with accuracy metric

In [671]:
ann.fit(x_train_processed, y_train, batch_size = 16, epochs = 100) #default batch size with 100 epochs.

Epoch 1/100
14/14 [==============================] - 0s 1ms/step - loss: 0.7019 - accuracy: 0.5607
Epoch 2/100
14/14 [==============================] - 0s 2ms/step - loss: 0.6819 - accuracy: 0.5935
Epoch 3/100
14/14 [==============================] - 0s 1ms/step - loss: 0.6652 - accuracy: 0.6542
Epoch 4/100
14/14 [==============================] - 0s 1ms/step - loss: 0.6493 - accuracy: 0.6822
Epoch 5/100
14/14 [==============================] - 0s 1ms/step - loss: 0.6340 - accuracy: 0.6822
Epoch 6/100
14/14 [==============================] - 0s 1ms/step - loss: 0.6209 - accuracy: 0.7196
Epoch 7/100
14/14 [==============================] - 0s 1ms/step - loss: 0.6085 - accuracy: 0.7103
Epoch 8/100
14/14 [==============================] - 0s 1ms/step - loss: 0.5980 - accuracy: 0.7056
Epoch 9/100
14/14 [==============================] - 0s 1ms/step - loss: 0.5880 - accuracy: 0.7056
Epoch 10/100
14/14 [==============================] - 0s 1ms/step - loss: 0.5793 - accuracy: 0.7103
Epoch 11/

In [672]:
y_pred = ann.predict(x_test_processed) #look at the results for the ann, 
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 1]
 [1 1]
 [0 0]
 [0 1]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [1 1]
 [1 0]
 [0 0]
 [1 0]
 [1 1]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [1 0]
 [0 1]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 1]
 [0 0]
 [0 0]
 [1 1]
 [0 0]
 [0 0]
 [0 0]
 [1 1]
 [0 1]
 [1 1]
 [0 1]
 [0 0]
 [1 1]
 [1 0]
 [0 0]
 [0 1]
 [0 0]
 [0 1]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 0]
 [0 1]
 [0 0]
 [0 0]
 [1 1]
 [1 1]
 [0 0]
 [0 1]]


In [673]:
from sklearn.metrics import confusion_matrix, accuracy_score #print confusion matrix and accuracy score!
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[41  4]
 [17 10]]


0.7083333333333334

In [674]:
#This model created ok results, with a decent accuracy score but still produced quite a lot of FP and i think it would need a bigger
#dataset to make it more accurate.
#This result however shows good accuracy just a shame about the number of FP which didnt decrease from 18-25 despite tuning efforts.

In [675]:
def test_no_nulls(data):
    
    # if data is numpy array, handle accordingly
    if isinstance(data, (np.ndarray)):
        assert not np.isnan(np.min(data))
    
    # if not np array, assume data is pandas dataframe
    else:
        assert data.isna().sum().sum() == 0 

In [676]:
# run null data unit test on both training and test data
test_no_nulls(x_train_processed)
test_no_nulls(x_test_processed)

In [677]:
#I enjoyed this module but the dataset chosen made it very challenging and not applicable to the modules i learned on Udemy due to the 
#added complexity of OHE due to the strings. This made the whole task take 30+ hours to complete and was very challenging.
#I hope this is enough to pass and demonstrate my knowledge even if it wasnt perfect! Thank you.